In [34]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

! pip install mlxtend

# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

you can refer to this article to find out about apriori and association rule:
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here: 
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [35]:
# load the data set and show the first five transaction
import pandas as pd
url = "https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv"
df = pd.read_csv(url)
first_five_transactions = df.head(5)
print(first_five_transactions)


        0       1     2       3       4       5       6
0   Bread    Wine  Eggs    Meat  Cheese  Pencil  Diaper
1   Bread  Cheese  Meat  Diaper    Wine    Milk  Pencil
2  Cheese    Meat  Eggs    Milk    Wine     NaN     NaN
3  Cheese    Meat  Eggs    Milk    Wine     NaN     NaN
4    Meat  Pencil  Wine     NaN     NaN     NaN     NaN


# Get the set of product that has been purchased


Get the unique product that has been purchased

In [36]:
unique_products = set(df.stack().dropna().unique())
print(unique_products)

{'Wine', 'Eggs', 'Milk', 'Meat', 'Cheese', 'Pencil', 'Bread', 'Bagel', 'Diaper'}


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [37]:
# Create an itemset based on the products
itemsets = [set(row) for _, row in df.iterrows()]

# Encoding the feature
encoded_df = pd.DataFrame(0, columns=df["1"].unique(), index=range(1, len(itemsets) + 1))
for i, itemset in enumerate(itemsets, start=1):
    encoded_df.loc[i, itemset] = 1


C:\Users\Andrew Lin\AppData\Local\Temp\ipykernel_24484\3133737778.py:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  encoded_df.loc[i, itemset] = 1
C:\Users\Andrew Lin\AppData\Local\Temp\ipykernel_24484\3133737778.py:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  encoded_df.loc[i, itemset] = 1
C:\Users\Andrew Lin\AppData\Local\Temp\ipykernel_24484\3133737778.py:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  encoded_df.loc[i, itemset] = 1
C:\Users\Andrew Lin\AppData\Local\Temp\ipykernel_24484\3133737778.py:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  encoded_df.loc[i, itemset] = 1
C:\Users\Andrew Lin\AppData\Local\Temp\ipykernel_24484\3133737778.py:7: FutureWarning: Passing a set as an indexer is deprecated

In [38]:
# Create a new DataFrame from the encoded features
new_df = pd.DataFrame(encoded_df)

# Show the new DataFrame
print(new_df.head())


   Wine  Cheese  Meat  Pencil  Bread  Diaper  Eggs  NaN  Bagel  Milk
1     1       1     1       1      1       1     1    0      0     0
2     1       1     1       1      1       1     0    0      0     1
3     1       1     1       0      0       0     1    1      0     1
4     1       1     1       0      0       0     1    1      0     1
5     1       0     1       1      0       0     0    1      0     0


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

In [39]:
new_df = df_encoded.drop(columns=["NaN"], errors="ignore")
new_df = df_encoded.iloc[:, 1:]
print(new_df.head())


   Bread  Cheese  Diaper   Eggs  Meat   Milk  Pencil  Wine    nan
0   True    True    True   True  True  False    True  True  False
1   True    True    True  False  True   True    True  True  False
2  False    True   False   True  True   True   False  True   True
3  False    True   False   True  True   True   False  True   True
4  False   False   False  False  True  False    True  True   True


## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products. 
For this case study, we will min_support=0.2

In [40]:
from mlxtend.frequent_patterns import apriori, association_rules
frequent_itemsets = apriori(df_encoded, min_support=0.2, use_colnames=True)
print(frequent_itemsets.sort_values(by='support', ascending=False))

     support              itemsets
9   0.869841                 (nan)
1   0.504762               (Bread)
2   0.501587              (Cheese)
6   0.501587                (Milk)
5   0.476190                (Meat)
8   0.438095                (Wine)
4   0.438095                (Eggs)
0   0.425397               (Bagel)
37  0.409524           (Milk, nan)
3   0.406349              (Diaper)
19  0.396825          (Bread, nan)
26  0.393651         (nan, Cheese)
35  0.368254           (nan, Meat)
7   0.361905              (Pencil)
12  0.336508          (nan, Bagel)
32  0.336508           (Eggs, nan)
22  0.323810        (Meat, Cheese)
40  0.317460           (Wine, nan)
28  0.317460         (nan, Diaper)
23  0.304762        (Milk, Cheese)
21  0.298413        (Eggs, Cheese)
10  0.279365        (Bread, Bagel)
16  0.279365         (Milk, Bread)
25  0.269841        (Wine, Cheese)
39  0.266667         (Pencil, nan)
29  0.266667          (Eggs, Meat)
34  0.250794          (Wine, Meat)
33  0.244444        

Then, we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [11]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.6)
print(rules[['antecedents', 'consequents', 'antecedent support', 'consequent support', 'support', 'confidence', 'lift', 'leverage', 'conviction']])


       antecedents consequents  antecedent support  consequent support  \
0          (Bagel)     (Bread)            0.425397            0.504762   
1          (Bagel)       (nan)            0.425397            0.869841   
2          (Bread)       (nan)            0.504762            0.869841   
3           (Eggs)    (Cheese)            0.438095            0.501587   
4           (Meat)    (Cheese)            0.476190            0.501587   
5         (Cheese)      (Meat)            0.501587            0.476190   
6           (Milk)    (Cheese)            0.501587            0.501587   
7         (Cheese)      (Milk)            0.501587            0.501587   
8           (Wine)    (Cheese)            0.438095            0.501587   
9         (Cheese)       (nan)            0.501587            0.869841   
10        (Diaper)       (nan)            0.406349            0.869841   
11          (Eggs)      (Meat)            0.438095            0.476190   
12          (Eggs)       (nan)        

Provide explanation about __antecedent support__, __consequent support__, __support__, __confidence__, __lift__, __leverage__ and __conviction__

In [ ]:
Antecedent Support: refers to the proportion of transactions in the 
dataset that includes the antecedent 
(located on the left-hand side of the rule).

Consequent Support: 
indicates the proportion of transactions in the dataset that includes the 
consequent 
(located on the right-hand side of the rule).


Support:
is the proportion of transactions in the dataset that contains 
both the antecedent and the consequent.


Confidence:
represents the conditional probability of the consequent 
given the antecedent, measuring the likelihood that the rule is true.


Lift:
measures how much more likely the consequent is given the antecedent 
compared to when the antecedent and consequent are independent.


Leverage:
quantifies the difference between the observed frequency of the 
antecedent and consequent appearing together and the frequency 
expected under independence.


Conviction:
gauges the ratio of the expected frequency of the antecedent 
occurring without the consequent to the observed frequency of incorrect 
predictions.